# Kriging Compute Task

This notebook demonstrates how to run kriging compute tasks using the `evo-compute` package.

Kriging is a geostatistical interpolation technique that estimates values at unsampled locations
using weighted averages of nearby known values, based on a variogram model of spatial correlation.

## Authentication

First, authenticate using the `ServiceManagerWidget`:

In [1]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="core-compute-tasks-notebooks",
    base_uri="https://qa-ims.bentley.com",
    discovery_url="https://int-discover.test.api.seequent.com",
    cache_location="notebook-data",
).login()

ServiceManagerWidget(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR…

## Example 1: Run Kriging on Existing Objects

This example shows how to run kriging using existing geoscience objects (source pointset, target grid, and variogram).

### Load the Source PointSet and Target Grid

In [2]:
from evo.notebooks import display_object_links
from evo.objects.typed import PointSet, RegularMasked3DGrid

# Replace with your actual object references
environment = manager.get_environment()
prefix = (
    f"{environment.hub_url}/geoscience-object/orgs/{environment.org_id}/workspaces/{environment.workspace_id}/objects"
)

source_pointset_ref = f"{prefix}/9100d7dc-44e9-4e61-b427-159635dea22f?version=1766012988078917061"
variogram_ref = f"{prefix}/72cd9b83-90f4-4cb0-9691-95728e3f9cbb?version=1765856020219372248"
target_grid_ref = f"{prefix}/df9c3705-c82e-4f57-af94-b3346b5d58cf?version=1765929914693546924"

# Load the source pointset
source_pointset = await PointSet.from_reference(manager, source_pointset_ref)
display_object_links(source_pointset, label="Source PointSet")
print(f"Source pointset: {source_pointset.name}")

# Load the target grid
target_grid = await PointSet.from_reference(manager, target_grid_ref)
display_object_links(target_grid, label="Target Grid")
print(f"Target grid: {target_grid.name}")

Source pointset: Ag_LMS1 - Ag_ppm Values


Target grid: Block Model - LMS1 Kr - point kriging


### Run Kriging Task

> **Note:** If the imports below fail, you may need to add the local source packages to your Python path. Run the cell below first.

In [3]:
# Setup for local development source - run this cell FIRST if you get import errors
# You may need to restart your kernel after running this cell for the first time
import sys

# Remove any cached evo.compute modules to force reimport from local source
mods_to_remove = [key for key in list(sys.modules.keys()) if key.startswith('evo.compute')]
for mod in mods_to_remove:
    del sys.modules[mod]

local_paths = [
    r"C:\Source\evo-python-sdk\packages\evo-compute\src",
    r"C:\Source\evo-python-sdk\packages\evo-objects\src",
    r"C:\Source\evo-python-sdk\packages\evo-sdk-common\src",
]
for path in local_paths:
    if path not in sys.path:
        sys.path.insert(0, path)

print("Local source paths configured - restart kernel if you still see import errors")

Local source paths configured - restart kernel if you still see import errors


In [4]:
from evo.compute.tasks import (
    run_kriging,
    KrigingParameters,
    Target,
    OrdinaryKriging,
    KrigingSearch,
    Ellipsoid,
    EllipsoidRanges,
    Rotation,
)
from evo.notebooks import FeedbackWidget

# Create kriging parameters
kriging_params = KrigingParameters(
    source=source_pointset.locations.attributes["Ag_ppm Values"],
    target=Target.new_attribute(object=target_grid, attribute_name="kriged_grade"),
    kriging_method=OrdinaryKriging(),
    variogram=variogram_ref,
    neighborhood=KrigingSearch(
        ellipsoid=Ellipsoid(
            ellipsoid_ranges=EllipsoidRanges(major=200.0, semi_major=150.0, minor=100.0),
            rotation=Rotation(dip_azimuth=0.0, dip=0.0, pitch=0.0),
        ),
        max_samples=20,
    ),
)

# Run the kriging task with progress feedback
print("Submitting kriging task...")
fb = FeedbackWidget("Kriging task")
result = await run_kriging(manager, kriging_params, fb=fb)

print(f"\nKriging completed!")
print(f"Message: {result.message}")
display_object_links(result.target.reference, label="Kriging Result")

Submitting kriging task...



Kriging completed!
Message: Kriging completed.


## Example 2: Create Objects and Run Kriging

This example shows how to create the input objects from scratch and then run kriging.

### Create the Source PointSet

In [ ]:
import uuid
import numpy as np
import pandas as pd
from evo.objects.typed import EpsgCode, PointSet, PointSetData

# Generate sample point data
n_points = 100
np.random.seed(42)

# Create points in a 1000x1000x100 domain
x = np.random.uniform(0, 1000, n_points)
y = np.random.uniform(0, 1000, n_points)
z = np.random.uniform(0, 100, n_points)

# Create an elevation attribute (z + some noise)
elevation = z + np.random.normal(0, 5, n_points)

# Create pointset using PointSetData
pointset_data = PointSetData(
    name=f"Sample Source Points - {uuid.uuid4()}",
    coordinate_reference_system=EpsgCode(32632),
    locations=pd.DataFrame({"x": x, "y": y, "z": z, "elevation": elevation}),
)

# Create the pointset object
source_pointset_created = await PointSet.create(manager, pointset_data)

print(f"Created source pointset: {source_pointset_created.name}")
display_object_links(source_pointset_created, label="Source PointSet")

### Create a Variogram

Note: There will be a typed object for variograms in a future release. For now, we use the low-level ObjectAPIClient.

In [ ]:
from evo.objects import ObjectAPIClient

object_client = ObjectAPIClient.from_context(manager)

# Define a spherical variogram model
variogram_metadata = {
    "schema": "/objects/variogram/1.1.0/variogram.schema.json",
    "name": "Sample Variogram",
    "nugget": 0.1,
    "sill": 1.0,
    "is_rotation_fixed": True,
    "structures": [
        {
            "variogram_type": "spherical",
            "contribution": 0.9,
            "anisotropy": {
                "ellipsoid_ranges": {"major": 200.0, "semi_major": 200.0, "minor": 100.0},
                "rotation": {"dip_azimuth": 0.0, "dip": 0.0, "pitch": 0.0},
            },
        }
    ],
    "attribute": "elevation",
    "domain": "all",
}

variogram = await object_client.create_geoscience_object(
    path=f"/variograms/sample-variogram-{uuid.uuid4()}.json",
    object_dict=variogram_metadata,
)

print(f"Created variogram: {variogram.name}")
display_object_links(variogram, label="Variogram")

### Create the Target Grid

In [ ]:
from evo.objects.typed import Point3, RegularMasked3DGrid, RegularMasked3DGridData, Size3d, Size3i
from evo.objects.typed import Rotation as GridRotation

# Define grid dimensions
nx, ny, nz = 20, 20, 10  # Number of cells in each direction
cell_size = 50.0  # Size of each cell

# Create a mask for the grid (all cells active in this example)
total_cells = nx * ny * nz
mask = np.ones(total_cells, dtype=bool)

# Optionally, mask out some cells to create a more interesting shape
for zi in range(nz // 2):
    for yi in range(ny // 2):
        for xi in range(nx // 2):
            idx = xi + yi * nx + zi * nx * ny
            mask[idx] = False

# Create masked grid using RegularMasked3DGridData
grid_data = RegularMasked3DGridData(
    name=f"Target Masked Grid - {uuid.uuid4()}",
    coordinate_reference_system=EpsgCode(32632),
    origin=Point3(0, 0, 0),
    size=Size3i(nx, ny, nz),
    cell_size=Size3d(cell_size, cell_size, cell_size),
    rotation=GridRotation(0, 0, 0),
    mask=mask,
    cell_data=None,  # No attributes yet, kriging will add them
)

# Create the grid object
target_grid_created = await RegularMasked3DGrid.create(manager, grid_data)

print(f"Created target grid: {target_grid_created.name}")
print(f"  Total cells: {nx} x {ny} x {nz} = {total_cells}")
print(f"  Active cells: {int(mask.sum())}")
print(f"  Bounding box: {target_grid_created.bounding_box}")
display_object_links(target_grid_created, label="Target Grid")

### Run Kriging on Created Objects

In [ ]:
from evo.compute.tasks import (
    run_kriging,
    KrigingParameters,
    Target,
    OrdinaryKriging,
    KrigingSearch,
    Ellipsoid,
    EllipsoidRanges,
    Rotation,
)
from evo.notebooks import FeedbackWidget

# Create kriging parameters using typed Attribute access
# source_pointset_created.locations.attributes["elevation"] gives us an Attribute object
kriging_params = KrigingParameters(
    source=source_pointset_created.locations.attributes["elevation"],
    target=Target.new_attribute(object=target_grid_created, attribute_name="kriged_elevation"),
    kriging_method=OrdinaryKriging(),
    variogram=variogram,
    neighborhood=KrigingSearch(
        ellipsoid=Ellipsoid(
            ellipsoid_ranges=EllipsoidRanges(major=134.0, semi_major=90.0, minor=40.0),
            rotation=Rotation(dip_azimuth=100.0, dip=65.0, pitch=75.0),
        ),
        max_samples=20,
    ),
)

# Run the kriging task
print("Submitting kriging task...")
fb = FeedbackWidget("Kriging task")
result = await run_kriging(manager, kriging_params, fb=fb)

print(f"\nKriging completed!")
print(f"Message: {result.message}")
display_object_links(result.target.reference, label="Kriging Result")
